## Load Data

In [48]:
from opencc import OpenCC
import re
import pandas
import emoji
import jieba

In [25]:
from pymongo import MongoClient
client = MongoClient()

In [26]:
db = client.test
trData = db.sampleset

In [27]:
import pandas as pd

In [28]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
#df_trData.head()

,_id,author,content,copyrightSource,id,last_update,pubSource,pubTime,title,url
0,5a826ea13ed17ebe58f25c61,丘份碗66,【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄...,丘份碗66,-9223352888553609071,2018-01-29 22:48:53,新浪微博,1517050815,丘份碗66,http://weibo.com/6142925293/G0mQ3azgA
1,5a826ea13ed17ebe58f25c62,asd紫衣,这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广...,asd紫衣,-9223363810167202147,2018-01-15 09:29:57,新浪微博,1515967706,asd紫衣,http://weibo.com/6276804601/FEq78giV6
2,5a826ea13ed17ebe58f25c63,故事与酒缸,【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰...,故事与酒缸,-9223357131622972986,2018-01-11 01:31:56,新浪微博,1515597336,故事与酒缸,http://weibo.com/5759812287/FDLHA0DtI
3,5a826ea13ed17ebe58f25c64,Caroline_Nine,消防车🚒//@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑...,Caroline_Nine,-9223330358968196918,2018-01-20 16:36:19,新浪微博,1516422278,Caroline_Nine,http://weibo.com/1875936727/FFdIjvCCL
4,5a826ea13ed17ebe58f25c65,qq34314152,【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到...,qq34314152,-9223311817460417728,2018-01-21 07:46:52,新浪微博,1516469637,qq34314152,http://weibo.com/2387005072/FFiSHoepC


In [29]:
df_trData = df_trData[~df_trData['content'].isnull()]

In [30]:
df_content = df_trData[['content']]

In [31]:
def extract_emojis(str):
      return ''.join(c for c in str if c not in emoji.UNICODE_EMOJI)

In [32]:
def strQ2B(ustring):
    """把字符串全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        # 全角字符unicode编码从65281~65374 （十六进制 0xFF01 ~ 0xFF5E）
        # 半角字符unicode编码从33~126 （十六进制 0x21~ 0x7E）
        # 而且除空格外,全角/半角按unicode编码排序在顺序上是对应的
        if inside_code==0x3000:
            # 空格 全角为0x3000 半角为0x0020
            inside_code=0x0020
        elif inside_code == 0x3002:
            # 中文句号
            inside_code=0x2e
        else:
            inside_code-=0xfee0
        if inside_code<0x0020 or inside_code>0x7e:      
            rstring += uchar
        else:
            rstring += chr(inside_code)
    return rstring

In [33]:
#df_content['content1'] = df_content.apply(lambda row: strQ2B(row['content1']), axis = 1)

## Merge to One String

In [34]:
df_list = df_content['content'].tolist()

In [35]:
#合併
df_all_in_one = '\n'.join(df_list)
df_all_in_one = ''.join(df_all_in_one.split())

In [36]:
df_all_in_one[0:500]

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市。“这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了//@张若昀:一杯不会止一杯【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。消防车🚒//@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑出猪声[允悲]//@梨园西池水:哈哈哈哈哈哈哈哈哈哈//@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……[47]【动画电影集散地原创】の《The'

In [37]:
#繁簡/大小寫
openCC = OpenCC('t2s')
df_all_in_one = openCC.convert(df_all_in_one).lower()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sophia.gao/.pyenv/versions/miniconda2-latest/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-5f6e8518b47c>", line 3, in <module>
    df_all_in_one = openCC.convert(df_all_in_one).lower()
  File "/Users/sophia.gao/Workspace/JupyterNotebook/NLP/opencc-python-master/opencc/opencc.py", line 71, in convert
    result.append(self._convert(split_string_list[i], self._dict_chain_data))
  File "/Users/sophia.gao/Workspace/JupyterNotebook/NLP/opencc-python-master/opencc/opencc.py", line 101, in _convert
    tree = StringTree(self._convert("".join(tree.inorder()), c_dict, True))
  File "/Users/sophia.gao/Workspace/JupyterNotebook/NLP/opencc-python-master/opencc/opencc.py", line 93, in _convert
    tree.convert_tree(c_dict)
  File "/Users/sophia.gao/Workspace/JupyterNotebook/NLP/opencc-python-master/opencc/opencc.py", line

KeyboardInterrupt: 

In [38]:
df_all_in_one[8000:10000]

"EPS0.14元,同比增...t.cn/zOllifE乐视网停牌9个月了，贾跃亭也走了，乐视网的朋友也早在危机之前跳到了万达信息，看近期的公告，乐视网已经做好复盘准备工作，就等待jgc一声令下了。复牌不远了，复牌几个跌停板大家猜的很尽兴。多家基金公司给出仅为3.91元左右的估值，停牌的收盘价是15.33元，这家四分三，还是四分之一了。估算...http://m.weibo.cn/client/version今天和我妈去街对面的小火锅店吃饭吃完回来过长城花园门前的十字路口斑马线两旁还闪着黄色信号灯我妈说现在银川厉害了过有摄像头的斑马线都要让行人的然后拉着我说过过过肯定都得停已经走到马路中心一辆车完全不带任何减速开过来然后我站住了…当时瞬间的想法是跑两步也肯定被撞不动也是被撞...http://m.weibo.cn/client/version在日益倡导环保的今天，混合动力的车也越来越普及。这款混合动力的电动车由一家以研发新能源为主的公司研发。车身上配有三块太阳能光电板和一个396W的电池组，足够日常的上班和买菜需要。宁波鄞州万达车展，1月6号到14号要兼职9人，8女1男，早九晚十(正式一般十点开始），连做。女生负责发传单，收集资料，男生负责擦车，150一天，外加20饭补。另外还需要有单反在拍照人员，不用专业，250一天，所有岗位轮休，工作不累，要求简单，有时间的兼职来报名。联系微信电话同15757830029....http://m.weibo.cn/client/version受益于低利率和通货膨胀的影响！不过伴随着东盟计划2018年取消该地区各成员国间的进口关税：越南车市需求不断增长，越南本土车企将面临国外车企的竞争，【越南车市需求强劲国外车企或成大赢家】据《日本经济新闻》报道http://t.cn/R6tDNm2低学历宝妈---CICI92年，25岁在从事微商之前在外打工4年，初中学历，换过无数的工作2014年22岁的CICI用一个月的房租450元拿了5个美痘人正式成为向日葵！从最低级别到官方，一赚到钱就升级，这样钱滚钱，到现在房产多套有车有工作室还有存款！下午看完电影去万达等基友考完六级约饭[跪了]基友看到我说我今天画个妆跟被打了一样。。。手残怪我咯[跪了]去了个音乐餐厅，[bm讨论]#属于我的金球时刻#以为是西餐。。。[跪了]然而打开菜单是干锅烧烤烤鱼。。

In [39]:
sent_pattern = u"[\u0030-\u0039\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff\u2000-\u2025\u2027\u206f\u3000-\u3001\u3003-\u3009\u3012-\u303f\uff00\uff02-\uff1f\uff20-\uffef]+[\u3002|\uff1f|\uff01|\u3010|\u3011|\u2026]+"
tag_pattern = u"\u0023+[\w\W\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\u0023"
id_pattern = u"\u0040+[\w\W\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\u003A"
url_pattern = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
url_pattern1 = "//(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
mail_pattern = "^\\s*\\w+(?:\\.{0,1}[\\w-]+)*@[a-zA-Z0-9]+(?:[-.][a-zA-Z0-9]+)*\\.[a-zA-Z]+\\s*$"
other_emoji = u"\[+[\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\]"

In [40]:
txt = "@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑出猪声[允悲]//@梨园西池水:哈哈哈哈哈哈哈哈哈哈//@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……[47]【动画电影集散地原创】の《The"
re.findall(sent_pattern, txt)

['微电影|',
 '无处可逃】',
 '杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……',
 '动画电影集散地原创】']

In [41]:
pa = tag_pattern+'|'+id_pattern+'|'+url_pattern+'|'+mail_pattern+'|'+other_emoji+'|'+url_pattern1

In [42]:
#忽略特定格式
handle = re.sub(pa,'',df_all_in_one)

In [43]:
handle_wo_emoji = extract_emojis(handle)

In [44]:
handle_wo_emoji[0:1000]

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市。“这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了张若昀:一杯不会止一杯【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。消防车菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的一个阿呆仔:学猪叫那个真笑出猪声梨园西池水:哈哈哈哈哈哈哈哈哈哈太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ【微电影|无处可逃】2017年10月31日上午，杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……【动画电影集散地原创】の《ThePolarExpress》〔20050121〕某天我们突然对自己曾经【BeeDragon打造完美VIP丰田皇冠】BeeDragon打造的标志性VIP风格。BeeDragon定制前保险杠，前大灯调色搭配HID雾灯，霸气的前脸，不怒自威。VIP改装车的效果总是令人惊叹。专业定制的气动悬挂，趴地下是必须的。外倾角并没有夸张到接受不了，这么一辆车停在路边，没有谁不会去多看两眼把。【MINIPACEMAN皮卡】MINI发布了PACEMANAdventureConcept概念车的官方图片。这款车仅会生产一台，目前MINI并没有计划将其量产。PACEMANAdventureConcept概念车是基于PACEMAN普通版车型改进而来的一款皮卡车型，该车采用绿色涂装，车顶配有探照灯以及置物架。人社部财政部近日联合印发了《企业年金办法》，自2018年2月1日起施行。企业缴费每年不超过工资总额的8%，企业和职工个人缴费合计不超过12%。具体由企业和职工协商确定。企业年金基金实行完全积累，为每个职工建立个人账户，按照国家有关规定投资运营；收益并入企业年金基金【科鲁兹两厢版】外观设计上，科鲁兹两厢版除了尾部设计外均与三厢版一致。全新的家族式分体格栅以及带有U型LED日间行车灯的前

In [52]:
def chinese_only(line):
    rule = re.compile(u"[^\u4e00-\u9fa5]")
    line = rule.sub('',line)
    return line

In [73]:
handle_chinese = chinese_only(handle_wo_emoji) 

In [74]:
seg_list = jieba.cut(handle_chinese, cut_all=True)
#print("Full Mode: " + "/ ".join(seg_list))  # 全模式

In [75]:
wf=open('result.txt','w+')
wf.write(",".join(seg_list))
#print "搜索引擎模式: ",", ".join(seg_list)
wf.close()

In [77]:
import re
inpath = 'result.txt'
lines = get_lines(inpath)
for idx, line in enumerate(lines):
    if idx > 10:
        break;
    print(line)

NameError: name 'get_lines' is not defined